# Data Exploration
this file is used to get an overview over the files, that are on the IMI server. Included .csv files are loaded and used to determine which probands can be used.

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2

import torch
import torch.nn.functional as F
import os
#from utils.plotting import flow2img, overlaySegment
#from utils.encoding import dice_coeff
#from utils.layers import warp, warpImage
#from utils.load_models import load_flownet2
#from utils.preprocessing import preprocessing_flownet

In [ ]:
# files are located here
path_to_data = "/share/data_ultraschall/compressions"

In [ ]:
len(os.listdir(path_to_data))

In [ ]:
# manually, but somewhat random selected to see the image quality
test_dir = os.path.join(path_to_data, "157")
test_dir

In [ ]:
# get two random images 
random_frames = np.random.choice(os.listdir(os.path.join(test_dir,"frames")), 2)
random_frames

In [ ]:
# load images and segmentations
img_1 = Image.open(os.path.join(test_dir,"frames",random_frames[0]))
img_2 = Image.open(os.path.join(test_dir,"frames",random_frames[1]))

seg_1 = Image.open(os.path.join(test_dir,"segmentations","1",random_frames[0]))
seg_2 = Image.open(os.path.join(test_dir,"segmentations","1",random_frames[1]))

In [ ]:
fig = plt.figure(figsize=(10,10))

plt.subplot(221)
plt.imshow(img_1)
plt.subplot(222)
plt.imshow(seg_1)
plt.subplot(223)
plt.imshow(img_2)
plt.subplot(224)
plt.imshow(seg_2)

In [ ]:
# images need to be normalized
print(np.array(img_1).max())

# segmentations have labels of 0,1,2 and will be divided by 100
print(np.array(seg_1).max())

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(overlaySegment(torch.from_numpy(np.array(img_1))/255,torch.from_numpy(np.array(seg_1))/100))
plt.subplot(122)
plt.imshow(overlaySegment(torch.from_numpy(np.array(img_2))/255,torch.from_numpy(np.array(seg_2))/100))

In order to get the right label for the segmentation, we need to divide the seg image by 100, as the labels are read in as 0, 100, and 200. the image needs to be devided by 255 to get float numbers

# Plotting from randomly from all folders
there ar around 3000 recordings and about 250.000 images with segmentations. There will need to be a selection of images taken. Maybe 1500 or 2000 images and segmentations overall. The distribution of the three markers should be 1/3 for each scan location. For this we need an overview over the different recordings. This is included in the quality.csv and sequences.csv files in the cocoAi filder...

In [ ]:
# load the quality overview
quality = pd.read_csv("~/MasterThesis/quality.csv")

# More info about the sequences
sequences = pd.read_csv("~/MasterThesis/sequences.csv")

In [ ]:
quality

In [ ]:
sequences

In [ ]:
# somewhat even
sequences['Leg'].value_counts()

There are some things, that need to be fulfilled in orer for the sequence to be used for training. The table below sums up all the different Columns and what needs to be fulfilled in order to use the ID for the training

| Quality Description | useable Index | Description |
|---|---|---|
|Vessel boundary| 1,2| Vessel partially cut of or fully visible during frames|
|LM configuration| 1,2| Landmarks are somewhat present for flow estimation|
|Vein contrast| 1,2 | some loss of contrast, but still visible|
|Artery contrast| 1,2 | some loss of contrast, but stil visible|
|shaprnes of Vein boundaries| 1,2 | up until poorly visible boundaries, We try to learn the overall flow|
|shaprnes of artery boundaries| 1,2 | up until poorly visible boundaries, We try to learn the overall flow|
|Overall gain| 1 | only good images are taken. Not too bright or too dark|
|artefacts| 1,2 | some aftefacts are present in the some frames|
| Movement Sequence| 1,2| Exclude "lot's" of movement for now. The other indicate some slow or fast compression of the vein|

In [ ]:
# filter based on the table above
useable = quality[
    #(quality["Vessel in Frame"] <3) &
    #(quality['LM configuration'] < 4) &
    #(quality['Vein contrast/cropping error'] < 3) &
    #(quality['Artery contrast'] < 3) &
    #(quality['Vein boundary'] < 3) &
    #(quality['Artery boundary'] < 3) &
    #(quality['Gain'] == 1) &
    #(quality['Artefacts'] < 3) &
    #(quality['Movement'] < 3) &
    (quality['Total'] < 20)
]
useable.rename(columns={'ID': 'Id'}, inplace=True)
useable

In [ ]:
# filter based on useable
#overview = sequences[sequences.Id.isin(useable.ID.tolist())]
overview = sequences.merge(useable, on='Id')
overview

In [ ]:
# Check how many of the selected IDs are available as data
overview_clean = overview[overview['Id'].astype(str).isin(os.listdir(path_to_data))]
overview_clean

In [ ]:
# get an overview. all seems to be fine.
print(overview_clean['Leg'].value_counts())
print(overview_clean['Device'].value_counts())
print(overview_clean['Datatype'].value_counts())

In [ ]:
# Are all IDs available as segmented data?
available_seg = []
for proband in overview_clean['Id'].values:
    proband_path = os.path.join(path_to_data,str(proband))
    if "segmentations" in os.listdir(proband_path):
        available_seg.append(proband)
len(available_seg)

In [ ]:
# filter based on segmentation available
overview_available = overview_clean[overview_clean['Id'].isin(available_seg)]
overview_available

In [ ]:
# sest random seed so we can come back and examine the data
np.random.seed(42)

In [ ]:
# choose 4 to visualize
random_selected_ids = np.random.choice(overview_available['Id'].values, size=6)
random_selected_ids

In [ ]:
# simple function to make the plotting easier
def create_plots(img_path, seg_path):
    img = Image.open(img_path)#/255 # normalize
    seg = Image.open(seg_path)#/100 # create labels
    
    return overlaySegment(torch.from_numpy(np.array(img))/255,torch.from_numpy(np.array(seg))/100)

In [ ]:
plots = []
for proband in random_selected_ids:
    proband_path = os.path.join(path_to_data,str(proband))
    
    # sort all images, to select fixed and moving image
    # the fixed will be randomly selected
    # and the moving will be X images further or before
    all_files = sorted(os.listdir(os.path.join(proband_path,"frames")))
    rand_idx = np.random.(0,lrandinten(all_files), size=2)
    file_idx = []
    for idx in rand_idx:
        
        # The time intervall atm is 4 images
        moving_idx = idx + 4
        if moving_idx > len(all_files)-1:
            moving_idx = idx - 4
            if moving_idx < 0:
                moving_idx = 0

        file_idx.append([all_files[idx],all_files[moving_idx]])
        
    plots.append([
        create_plots(os.path.join(proband_path,"frames",file_idx[0][0]),
                    os.path.join(proband_path,"segmentations","1",file_idx[0][0])),
        create_plots(os.path.join(proband_path,"frames",file_idx[0][1]),
                    os.path.join(proband_path,"segmentations","1",file_idx[0][1]))
    ])
        

# Visualizations

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[0][0])
plt.subplot(122)
plt.imshow(plots[0][1])

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[1][0])
plt.subplot(122)
plt.imshow(plots[1][1])

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[2][0])
plt.subplot(122)
plt.imshow(plots[2][1])

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[3][0])
plt.subplot(122)
plt.imshow(plots[3][1])

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[4][0])
plt.subplot(122)
plt.imshow(plots[4][1])

In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(121)
plt.imshow(plots[5][0])
plt.subplot(122)
plt.imshow(plots[5][1])

# Processing
Next step is to load all images and save them in a .pth file, to load them later. This will be done in the US_data_processing file

In [ ]:
overview_available.to_csv("~/MasterThesis/available_US_probands.csv")

# Flow Estimation Tests for visualization. 
This part of the notebook includes some tests on how to vizualize the optical flow fields and what is the best way to warp segementations. 

Most of the tests failed :(

In [ ]:
baseline = cv2.optflow.DualTVL1OpticalFlow_create()

In [ ]:
np.array(img_1)[:,:,np.newaxis].shape

In [ ]:
in1 = np.array(img_1)[:,:,np.newaxis].astype(np.float32) / 255
in2 = np.array(img_2)[:,:,np.newaxis].astype(np.float32) / 255
flow = baseline.calc(in1,in2,None)

In [ ]:
flow.shape

In [ ]:
plt.subplot(131)
plt.imshow(in1)
plt.subplot(132)
plt.imshow(flow2img(flow))
plt.subplot(133)
plt.imshow(in2)

In [ ]:
#flow = torch.from_numpy(flow)
seg = torch.from_numpy(np.asarray(seg_1)).long().contiguous() /200
seg.shape

In [ ]:
flow=torch.from_numpy(flow)

In [ ]:
warped = F.grid_sample(seg.unsqueeze(0).unsqueeze(0),flow.unsqueeze(0).reshape(1,150,150,2),mode='nearest',align_corners=False)
warped.shape

In [ ]:
#warped_1 = warp(seg.unsqueeze(0).unsqueeze(0), flow.view(1,2,150,150))
warped_2 = warpImage(seg.view(1,1,150,150).float(), flow.view(1,2,150,150))

In [ ]:
def overlaySegment(gray1,seg1,flag=False):
    H, W = seg1.squeeze().size()
    colors=torch.FloatTensor([0,0,0,199,67,66,225,140,154,78,129,170,45,170,170,240,110,38,111,163,91,235,175,86,202,255,52,162,0,183]).view(-1,3)/255.0
    segs1 = labelMatrixOneHot(seg1.unsqueeze(0),3)

    
    seg_color = torch.mm(segs1.view(3,-1).t(),colors[:3,:]).view(H,W,3)
    alpha = torch.clamp(1.0 - 0.5*(seg1>0).float(),0,1.0)

    overlay = (gray1*alpha).unsqueeze(2) + seg_color*(1.0-alpha).unsqueeze(2)
    if(flag):
        plt.imshow((overlay).numpy())
        plt.show()
    return overlay

def labelMatrixOneHot(segmentation, label_num):
    B, H, W = segmentation.size()
    values = segmentation.view(B,1,H,W).expand(B,label_num,H,W).to(segmentation.device)
    linspace = torch.linspace(0, label_num-1, label_num).long().view(1,label_num,1,1).expand(B,label_num,H,W).to(segmentation.device)
    matrix = (values.float()==linspace.float()).float().to(segmentation.device)
    for j in range(2,matrix.shape[1]):
        matrix[0,j,:,:] = matrix[0,j,:,:]
    return matrix

In [ ]:
plt.subplot(121)
plt.imshow(overlaySegment(torch.from_numpy(in2).view(150,150),warped_2.view(150,150)))
plt.subplot(122)
plt.imshow(overlaySegment(torch.from_numpy(in1).view(150,150),seg.reshape(150,150)))

In [ ]:
warped_2.shape

In [ ]:
warped_2.max()

In [ ]:
plt.imshow(warped_2.squeeze())

# Verdict baseline
has difficulties.. Probably need to look at the gird_sample or warp method to have best results.

In [ ]:
flow = load_flownet2()

In [ ]:
moving = F.interpolate(torch.from_numpy(in1).view(1,1,150,150), size=(2*64,2*64)).view(2*64,2*64,1) 
fixed = F.interpolate(torch.from_numpy(in2).view(1,1,150,150), size=(2*64,2*64)).view(2*64,2*64,1)

print(moving.shape)
print(fixed.shape)

image = preprocessing_flownet(fixed, moving)
image.shape

In [ ]:
flow_out = flow(image).squeeze().detach().numpy().transpose(1,2,0)
flow_out.shape

In [ ]:
fig = plt.figure(figsize=(10,10))

plt.subplot(131)
plt.imshow(fixed)
plt.subplot(132)
plt.imshow(flow2img(flow_out))
plt.subplot(133)
plt.imshow(moving)

In [ ]:
seg.shape

In [ ]:
seg_inter = F.interpolate(seg.view(1,1,150,150), size=(128,128)).squeeze().long()
seg_inter.shape

In [ ]:
plt.imshow(overlaySegment(moving.squeeze(),seg_inter))

In [ ]:
warped = warpImage(seg_inter.view(1,1,128,128).float(), torch.from_numpy(flow_out).reshape(1,2,128,128))
warped_sampled = F.grid_sample(seg_inter.view(1,1,128,128).float(), torch.from_numpy(flow_out).reshape(1,128,128,2))

In [ ]:
plt.imshow(overlaySegment(moving.view(128,128),warped.squeeze()))

In [ ]:
plt.imshow(warped_sampled.squeeze())

These two do not work so well.. Let's try the cuda sampling one

In [ ]:
torch.cuda.is_available()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
crnt_dev = torch.cuda.current_device()

warped = warp(seg_inter.view(1,1,128,128).float().to(crnt_dev), torch.from_numpy(flow_out).reshape(1,2,128,128).to(crnt_dev)).cpu()

In [ ]:
plt.imshow(overlaySegment(fixed.view(128,128),warped.squeeze()))

The does not look like it is good...
